In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from load_data import load_K1, load_SE_A
from plotting_routines import generate_filename_fRG, change_prefix, plot_fRG
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rc('text', usetex=True)
plt.rc('font', family='serif')


path = "../Data/"
PC = "parquet_check_"
EA = "error_analysis_"

In [ ]:
filename = generate_filename_fRG(path, 1, 1, L_ini=1000, nODE=100)
filename_PC = change_prefix(path, PC, filename)
filename_EA = change_prefix(path, EA, filename)

In [ ]:
w_fRG, K1a_fRG, K1p_fRG, K1t_fRG = load_K1(filename)
w_PC , K1a_PC , K1p_PC , K1t_PC  = load_K1(filename_PC)

v_fRG, SE_fRG, A_fRG = load_SE_A(filename)
v_PC , SE_PC , A_PC  = load_SE_A(filename_PC)

In [ ]:
with h5py.File(path+filename_EA, 'r') as f:
    lambdas = np.array(f['lambdas'])
    norms_K1_fRG = np.array(f['norm_K1_fRG'])
    norms_K1_PC  = np.array(f['norm_K1_BSE'])
    K1_diff_abs = np.array(f['K1_diff_abs'])
    K1_diff_rel = np.array(f['K1_diff_rel'])
    
    norms_K2_fRG = np.array(f['norm_K2_fRG'])
    norms_K2_PC  = np.array(f['norm_K2_BSE'])
    K2_diff_abs = np.array(f['K2_diff_abs'])
    K2_diff_rel = np.array(f['K2_diff_rel'])
                            
    norms_S_fRG = np.array(f['norm_S_fRG'])
    norms_S_PC  = np.array(f['norm_S_SDE'])
    Sigma_diff_abs = np.array(f['Sigma_diff_abs'])
    Sigma_diff_rel = np.array(f['Sigma_diff_rel'])

In [ ]:




# real part of spin susceptibility (retarded comp. of K1a)
ax[0][0].plot(w_fRG, K1a_fRG[0].real, label='fRG')
ax[0][0].plot(w_PC , K1a_PC[0].real, label='BSE')

# imag part of spin susceptibility (retarded comp. of K1a = -advanced comp. of K1a)
ax[0][1].plot(w_fRG, -K1a_fRG[0].imag, label='fRG')
ax[0][1].plot(w_PC, -K1a_PC[0].imag, label='BSE')

#real part of the self energy
ax[1][0].plot(v_fRG, SE_fRG[0].real, label='fRG')
ax[1][0].plot(v_PC , SE_PC[0].real, label='BSE')
#imag part of the self energy
ax[1][1].plot(v_fRG, SE_fRG[0].imag, label='fRG')
ax[1][1].plot(v_PC , SE_PC[0].imag, label='BSE')

ax[0][0].set_ylabel(r'$\mathrm{Re}(\chi_\mathrm{sp}(\omega))$', fontsize=fs)
ax[0][1].set_ylabel(r'$\mathrm{Im}(\chi_\mathrm{sp}(\omega))$', fontsize=fs)
ax[1][0].set_ylabel(r'$\mathrm{Re}(\Sigma^R(\nu))$', fontsize=fs)
ax[1][1].set_ylabel(r'$\mathrm{Im}(\Sigma^R(\nu))$', fontsize=fs)

for i in range(2):
    ax[0][i].set_xlim(w_fRG[0], w_fRG[-1])
    ax[0][i].set_xlabel(r'$\omega/\Gamma$', fontsize=fs)
    
    ax[1][i].set_xlim(v_fRG[0], v_fRG[-1])
    ax[1][i].set_xlabel(r'$\nu/\Gamma$', fontsize=fs)
    
    ax[0][i].tick_params(axis='both', labelsize=fs-1)
    ax[1][i].tick_params(axis='both', labelsize=fs-1)

    ax[i][0].legend(fontsize=fs)
    

fig.tight_layout(pad=3.0)

plt.savefig("spin_susc@Lambda="+str(lambdas[0])+".pdf")


In [ ]:
fs = 18
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(36,12))

Deltas = (lambdas + 0.2)/2

ax[0][0].plot(Deltas, norms_K1_fRG, '.', label=r'$||\mathrm{fRG}||$')
ax[0][0].plot(Deltas, norms_K1_PC , '.', label=r'$||\mathrm{BS }||$')
ax[0][1].plot(Deltas, K1_diff_abs, '.', label=r'$||\mathrm{fRG-BS}||$')
ax[0][2].plot(Deltas, K1_diff_rel, '.', label=r'$\frac{||\mathrm{fRG-BS}||}{||\mathrm{fRG}||}$')

ax[1][0].plot(Deltas, norms_S_fRG, '.', label=r'$||\mathrm{fRG}||$')
ax[1][0].plot(Deltas, norms_S_PC , '.', label=r'$||\mathrm{SD }||$')
ax[1][1].plot(Deltas, Sigma_diff_abs, '.', label=r'$||\mathrm{fRG-SD}||$')
ax[1][2].plot(Deltas, Sigma_diff_rel, '.', label=r'$\frac{||\mathrm{fRG-SD}||}{||\mathrm{fRG}||}$')


ax[0][0].set_ylabel(r'$||\Gamma_{\mathrm{fRG/BS}}||$',fontsize=fs)
ax[0][1].set_ylabel(r'$||\Gamma_{\mathrm{fRG}-\mathrm{BS}}||$',fontsize=fs)
ax[0][2].set_ylabel(r'$||\Gamma_{\mathrm{fRG}-\mathrm{BS}}||/||\Gamma_{\mathrm{fRG}}||$',fontsize=fs)
ax[1][0].set_ylabel(r'$||\Sigma_{\mathrm{fRG/SD}}||$',fontsize=fs)
ax[1][1].set_ylabel(r'$||\Sigma_{\mathrm{fRG}-\mathrm{BS}}||$',fontsize=fs)
ax[1][2].set_ylabel(r'$||\Sigma_{\mathrm{fRG}-\mathrm{BS}}||/||\Sigma_{\mathrm{fRG}}||$',fontsize=fs)


#ax[0][0].set_ylim(1.8,2.4)
#ax[1][0].set_ylim(0.4,0.6)

for i in range(2):
    for j in range(3):
        ax[i][j].axvline(1, color='k', label=r'$U=\Delta$')
        ax[i][j].set_xlim(Deltas[0], Deltas[-1])
        ax[i][j].set_xscale("log")
        ax[1][j].set_xlabel(r'$\Delta$', fontsize = fs)
        ax[i][j].set_yscale("log")
        ax[i][j].legend(loc='lower right', fontsize=fs-4)
        ax[i][j].tick_params(axis='both', labelsize=fs-1)



plt.savefig("error_analysis_in_Delta_nODE=100.pdf")